In [4]:
import os
import pandas as pd
cwd = os.getcwd()
print(cwd)

/home/rraju


In [5]:
df = pd.read_csv('data/bookclub_events.csv')

/home/rraju/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.columns

Index(['Unnamed: 0', 'row_id', 'trial_time', 'event_name', 'event_time',
       'receipt_time', 'learner_id', 'session_id', 'book_code',
       'contents_code', 'library_source', 'media_fm'],
      dtype='object')

In [54]:
df['book_code'].describe()

count                 13508060
unique                   11557
top       BO-20170627173817922
freq                    512221
Name: book_code, dtype: object

In [55]:
df['learner_id'].describe()

count    2.364462e+07
mean     3.824910e+05
std      1.815107e+05
min      5.000000e+00
25%      2.348240e+05
50%      4.041670e+05
75%      5.137780e+05
max      6.669300e+05
Name: learner_id, dtype: float64

In [8]:
len(df)

23644617

In [10]:
len(df['row_id'].unique())

23644461

In [12]:
df['row_id'].describe()

count                                              23644617
unique                                             23644461
top       000000000004E4230000000000066B940000015E82461A...
freq                                                      4
Name: row_id, dtype: object

In [13]:
res = df['row_id'].value_counts()
print(res)

000000000004E4230000000000066B940000015E82461A90000000000000015E824EF021    4
000000000006FCAA000000000007CAFF0000015E8247DFB0000000000000015E824EF1EA    4
000000000006903A00000000000766490000015E59538F50000000000000015E824EF039    4
000000000005371D0000000000068B1F0000015E82461A90000000000000015E824EF02F    4
000000000006FCAA000000000007CAFF0000015E82462648000000000000015E824EF041    4
000000000006DBB8000000000007B4520000015E824612C0000000000000015E824EF02D    4
00000000000271B6000000000001B7E70000015E8247ACE8000000000000015E824EF1B5    4
000000000001593E00000000000339290000015E82467850000000000000015E824EF0B5    4
000000000006FCAA000000000007CAFF0000015E8247D3F8000000000000015E824EF1DE    4
000000000006903A00000000000766490000015E64B3C6A8000000000000015E824EF193    4
00000000000765B100000000000735E40000015E8246ED80000000000000015E824EF112    4
000000000001593E00000000000339290000015E82467850000000000000015E824EF0BB    4
000000000002AEFE000000000000C5130000015E8246D228000000000000015E

In [14]:
df[df['row_id'] == '000000000004E4230000000000066B940000015E82461A90000000000000015E824EF021']

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
18968667,18968668,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4
18968668,18968669,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4
18968709,18968710,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4
18968710,18968711,000000000004E4230000000000066B940000015E82461A...,1970-01-01,video_first_open,2017-09-14 21:23:06,2017-09-14 21:32:44,420756.0,002686229820170915061936,BO-20170821132107586,CT-20170821132107586,TL,MP4


In [16]:
df[df['row_id'] == '000000000006903A00000000000766490000015E64B3C6A8000000000000015E824EF197']

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
10708216,10708217,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA
10708217,10708218,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA
10708241,10708242,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA
10708242,10708243,000000000006903A00000000000766490000015E64B3C6...,1970-01-01,book_open,2017-09-09 03:34:17,2017-09-14 21:32:45,484937.0,002708098220170915062432,BO-20140726014638460,CT-20140726014638460,TBE,PDFA


In [17]:
len(df['trial_time'].unique())

1

In [18]:
len(df['event_name'].unique())

10

In [20]:
df['event_name'].unique()

array(['audio_close', 'audio_open', 'audio_first_open', 'book_close',
       'book_complete', 'book_first_open', 'book_open', 'video_close',
       'video_first_open', 'video_open'], dtype=object)

In [19]:
df['event_name'].value_counts()

video_close         5401557
video_open          4163399
book_open           4041230
book_close          4004299
book_first_open     2246649
video_first_open    1483684
book_complete       1438767
audio_open           421807
audio_close          407879
audio_first_open      35346
Name: event_name, dtype: int64

In [21]:
no_of_learners = len(df['learner_id'].unique())

264822

In [22]:
len(df['session_id'].unique())

2326130

In [25]:
df['session_id'].describe()

count                     23644617
unique                     2326130
top       002714403420170921203227
freq                          2343
Name: session_id, dtype: object

In [29]:
sessions = df['session_id'].value_counts()
sessions.to_csv('sessions.csv')

In [30]:
df[df['session_id'] == '002735577520170926212651']

,Unnamed: 0,row_id,trial_time,event_name,event_time,receipt_time,learner_id,session_id,book_code,contents_code,library_source,media_fm
390236,390237,000000000007CE7F000000000009EBC20000015EBE4328...,1970-01-01,audio_close,2017-09-26 12:57:06,2017-09-26 13:01:48,650178.0,002735577520170926212651,BO-20150417020356430,NaN,NaN,NaN
390346,390347,000000000007CE7F000000000009EBC20000015EBE2A41...,1970-01-01,audio_close,2017-09-26 12:29:54,2017-09-26 12:32:59,650178.0,002735577520170926212651,BO-20150417020356433,NaN,NaN,NaN
390358,390359,000000000007CE7F000000000009EBC20000015EBE2B1C...,1970-01-01,audio_close,2017-09-26 12:30:50,2017-09-26 12:36:36,650178.0,002735577520170926212651,BO-20150417020356522,NaN,NaN,NaN
390466,390467,000000000007CE7F000000000009EBC20000015EBE335D...,1970-01-01,audio_close,2017-09-26 12:39:51,2017-09-26 12:43:05,650178.0,002735577520170926212651,BO-20150417020356516,NaN,NaN,NaN
390552,390553,000000000007CE7F000000000009EBC20000015EBE3814...,1970-01-01,audio_close,2017-09-26 12:45:00,2017-09-26 12:48:03,650178.0,002735577520170926212651,BO-20150417020356388,NaN,NaN,NaN
390660,390661,000000000007CE7F000000000009EBC20000015EBE404E...,1970-01-01,audio_close,2017-09-26 12:53:59,2017-09-26 12:57:30,650178.0,002735577520170926212651,BO-20150417020356400,NaN,NaN,NaN
390747,390748,000000000007CE7F000000000009EBC20000015EBE2EFC...,1970-01-01,audio_close,2017-09-26 12:35:04,2017-09-26 12:38:11,650178.0,002735577520170926212651,BO-20150417020356424,NaN,NaN,NaN
390857,390858,000000000007CE7F000000000009EBC20000015EBE30D1...,1970-01-01,audio_close,2017-09-26 12:37:04,2017-09-26 12:42:09,650178.0,002735577520170926212651,BO-20150417020356519,NaN,NaN,NaN
391025,391026,000000000007CE7F000000000009EBC20000015EBE3EFA...,1970-01-01,audio_close,2017-09-26 12:52:32,2017-09-26 12:57:05,650178.0,002735577520170926212651,BO-20150417020356513,NaN,NaN,NaN
391183,391184,000000000007CE7F000000000009EBC20000015EBE49D7...,1970-01-01,audio_close,2017-09-26 13:04:24,2017-09-26 13:07:31,650178.0,002735577520170926212651,BO-20150417020356522,NaN,NaN,NaN


In [23]:
len(df['book_code'].unique())

11558

In [24]:
len(df['contents_code'].unique())

10121

In [42]:
user_item_df = df[['learner_id', 'book_code']]

In [44]:
user_item_df['learner_id'].value_counts()

417211.0    13224
342381.0    11911
404783.0    11626
388560.0    10352
639207.0     8826
503664.0     7501
339189.0     7431
419867.0     7262
480005.0     7169
430032.0     7081
453255.0     6361
441531.0     6300
310785.0     6226
192528.0     6022
509307.0     5715
492373.0     5545
472788.0     5323
650178.0     5137
472745.0     4863
637376.0     4748
585428.0     4730
398281.0     4572
445863.0     4151
336525.0     4104
640981.0     4077
252353.0     3921
403964.0     3893
502639.0     3712
259386.0     3700
637441.0     3697
            ...  
377744.0        1
377825.0        1
546800.0        1
547295.0        1
156839.0        1
376879.0        1
43093.0         1
204499.0        1
107859.0        1
156800.0        1
304206.0        1
550357.0        1
156725.0        1
549626.0        1
204777.0        1
43414.0         1
548918.0        1
43472.0         1
156649.0        1
156611.0        1
548294.0        1
303906.0        1
376685.0        1
376779.0        1
547622.0  

In [46]:
learner = df[df['learner_id'] == 417211]
no_of_books = len(learner['book_code'].unique())
print(no_of_books)

65


In [47]:
all_books = len(learner['book_code'])
print(all_books)

13224


In [48]:
df1 = df.groupby(['learner_id','book_code']).size().reset_index().rename(columns={0:'count'})

In [2]:
df1.head()

NameError: name 'df1' is not defined

In [51]:
df1[df1['learner_id'] == 417211]

,learner_id,book_code,count
1837661,417211.0,BO-20150417020356763,206
1837662,417211.0,BO-20150417020356766,208
1837663,417211.0,BO-20150417020356769,212
1837664,417211.0,BO-20150417020356772,206
1837665,417211.0,BO-20150417020356775,211
1837666,417211.0,BO-20150417020356778,208
1837667,417211.0,BO-20150417020356781,209
1837668,417211.0,BO-20150417020356784,197
1837669,417211.0,BO-20150417020356787,208
1837670,417211.0,BO-20150417020356790,202
